In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv('Social_Network_Ads.csv')
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype='int')
dataset.drop('User ID',axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [4]:
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
indep=dataset[['Gender_Male','Age','EstimatedSalary']]
dep=dataset[['Purchased']]

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
param_grid={'var_smoothing':np.logspace(0,-9,num=100)}
grid=GridSearchCV(GaussianNB(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=GaussianNB(), n_jobs=-1,
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             scoring='f1_weighted', verbose=3)

In [8]:
result=grid.cv_results_
grid_prediction=grid.predict(x_test)

In [9]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_prediction)
print('confusion matrix:\n',cm)

confusion matrix:
 [[75  4]
 [10 31]]


In [10]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_prediction)
print('The clf report :\n',clf_report)

The clf report :
               precision    recall  f1-score   support

           0       0.88      0.95      0.91        79
           1       0.89      0.76      0.82        41

    accuracy                           0.88       120
   macro avg       0.88      0.85      0.87       120
weighted avg       0.88      0.88      0.88       120



In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_prediction,average='weighted')
print('The f1_macro value for the best parameters {}:'.format(grid.best_params_),f1_macro)

The f1_macro value for the best parameters {'var_smoothing': 8.111308307896856e-09}: 0.8808622165169021


In [12]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.960481630132757

In [13]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.013815,0.003823,0.015548,0.003368,1.0,{'var_smoothing': 1.0},0.703041,0.660618,0.588177,0.758537,0.622874,0.666649,0.059797,100
1,0.013503,0.005519,0.015135,0.005853,0.811131,{'var_smoothing': 0.8111308307896871},0.729723,0.714286,0.588177,0.782059,0.622874,0.687424,0.071391,99
2,0.009970,0.003537,0.014625,0.005887,0.657933,{'var_smoothing': 0.657933224657568},0.739213,0.714286,0.618196,0.771493,0.609571,0.690552,0.065232,98
3,0.010276,0.003348,0.013568,0.002477,0.53367,{'var_smoothing': 0.533669923120631},0.723577,0.739213,0.618196,0.771493,0.609571,0.692410,0.066009,97
4,0.005159,0.004504,0.013431,0.002709,0.432876,{'var_smoothing': 0.43287612810830584},0.747056,0.739213,0.618196,0.771493,0.609571,0.697106,0.068834,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.005721,0.001404,0.013113,0.001270,0.0,{'var_smoothing': 2.310129700083158e-09},0.852966,0.840114,0.763089,0.946153,0.963889,0.873242,0.073723,10
96,0.008762,0.001669,0.010232,0.001387,0.0,{'var_smoothing': 1.873817422860387e-09},0.852966,0.840114,0.763089,0.946153,0.963889,0.873242,0.073723,10
97,0.004011,0.001798,0.012174,0.002457,0.0,{'var_smoothing': 1.519911082952933e-09},0.852966,0.840114,0.766471,0.946153,0.963889,0.873918,0.072717,9
98,0.005898,0.003451,0.009845,0.002988,0.0,{'var_smoothing': 1.2328467394420635e-09},0.852966,0.840114,0.766471,0.964286,0.963889,0.877545,0.076579,2
